# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,east london,-33.0153,27.9116,295.67,60,0,8.75,ZA,Mon Oct 2 10:51:49 2023
1,1,papatowai,-46.5619,169.4708,276.74,79,69,3.02,NZ,Mon Oct 2 10:51:55 2023
2,2,blackmans bay,-43.0167,147.3167,282.22,66,0,4.23,AU,Mon Oct 2 10:51:55 2023
3,3,stanley,54.8680,-1.6985,287.75,81,20,2.57,GB,Mon Oct 2 10:48:29 2023
4,4,polyarnyy,69.1989,33.4478,276.43,66,80,4.70,RU,Mon Oct 2 10:51:55 2023


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
#%%capture --no-display

# Configure the map plot
plot_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 0.5,
    color = "City"
)
# Display the map

plot_map

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [33]:
# Narrow down cities that fit criteria and drop any results with null values
vacation_df = city_data_df.loc[ (city_data_df['Max Temp']>(21+273.15)) 
                               &(city_data_df['Max Temp']<(27+273.15))
                               &(city_data_df['Wind Speed']<4.5)
                               &(city_data_df['Cloudiness']==0)]

# Drop any rows with null values
vacation_df.dropna(inplace = True)
# Display sample data
vacation_df.head()

C:\Users\zw\AppData\Local\Temp\ipykernel_2248\3850508527.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacation_df.dropna(inplace = True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
35,35,westport,41.1415,-73.3579,296.37,68,0,3.60,US,Mon Oct 2 10:50:03 2023
102,102,mastic beach,40.7668,-72.8521,295.43,78,0,4.47,US,Mon Oct 2 10:52:06 2023
127,127,holualoa,19.6228,-155.9522,295.06,79,0,3.13,US,Mon Oct 2 10:52:09 2023
142,142,bredasdorp,-34.5322,20.0403,295.98,44,0,3.73,ZA,Mon Oct 2 10:52:10 2023
146,146,hamilton,39.1834,-84.5333,298.70,72,0,1.54,US,Mon Oct 2 10:51:14 2023


### Step 3: Create a new DataFrame called `hotel_df`.

In [42]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City","Lat","Lng","Humidity","Country"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

C:\Users\zw\AppData\Local\Temp\ipykernel_2248\3313297143.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Lat,Lng,Humidity,Country,Hotel Name
0,east london,-33.0153,27.9116,60,ZA,
1,papatowai,-46.5619,169.4708,79,NZ,
2,blackmans bay,-43.0167,147.3167,66,AU,
3,stanley,54.8680,-1.6985,81,GB,
4,polyarnyy,69.1989,33.4478,66,RU,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [44]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": "1",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
east london - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
stanley - nearest hotel: Hotel 52
polyarnyy - nearest hotel: No hotel found
homer - nearest hotel: Driftwood Inn & Homer Seaside Lodges
puerto natales - nearest hotel: Dorotea Patagonia Hostel
port-aux-francais - nearest hotel: Keravel
valparaiso - nearest hotel: Ibis Hotel
west island - nearest hotel: Cocos Village Bungalows
korla - nearest hotel: Silver Star Hotel
motygino - nearest hotel: No hotel found
freeport - nearest hotel: The Freeport Inn and Marina
hermanus - nearest hotel: Aloe guest house
carnarvon - nearest hotel: No hotel found
beaufort west - nearest hotel: Matoppo Inn
iqaluit - nearest hotel: Frobisher Inn
ko pha ngan - nearest hotel: The Shore Resort
waitangi - nearest hotel: Hotel Chathams
tiksi - nearest hotel: Арктика
thompson - nearest hotel: Thompson Inn
nabire - nearest hotel: Hotel Anggrek
budva - nearest hotel:

,City,Lat,Lng,Humidity,Country,Hotel Name
0,east london,-33.0153,27.9116,60,ZA,No hotel found
1,papatowai,-46.5619,169.4708,79,NZ,No hotel found
2,blackmans bay,-43.0167,147.3167,66,AU,Villa Howden
3,stanley,54.8680,-1.6985,81,GB,Hotel 52
4,polyarnyy,69.1989,33.4478,66,RU,No hotel found
...,...,...,...,...,...,...
575,kill devil hills,36.0307,-75.6760,69,US,Mariner Days Inn & Suites
576,tra vinh,9.9347,106.3453,97,VN,No hotel found
577,tashir,41.1207,44.2846,76,AM,Voroncovk
578,hermansverk,61.1846,6.8502,71,NO,Sognefjord Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [46]:
%%capture --no-display

# Configure the map plot

hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols = "all"
)
# Display the map

hotel_map
